In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import string
import re 
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer

In [3]:
df = pd.read_csv('Data/complete_df.csv')
df

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30203,9iM7W1Dvl6Q,18.13.06,GLOW: Season 2 | Main Trailer [HD] | Netflix,Netflix,24,2018-06-11T14:30:11.000Z,"Netflix|""Trailer""|""movies""|""streaming""|""movies...",126958,2690,49,185,https://i.ytimg.com/vi/9iM7W1Dvl6Q/default.jpg,False,False,False,GLOW big or GLOW home. The girls are back in t...
30204,g4txv3A9ICQ,18.14.06,5 Seconds of Summer - No Roots (Alice Merton C...,BBCRadio1VEVO,10,2018-06-12T15:50:25.000Z,"5SOS|""5 Seconds of Summer""|""No Roots""|""Cover""|...",92541,12867,39,986,https://i.ytimg.com/vi/g4txv3A9ICQ/default.jpg,False,False,False,5SOS cover Alice Merton's No Roots in the BBC ...
30205,YQJmvXamKYg,18.14.06,Conway: People are bending to the will of Pres...,Fox News,25,2018-06-13T12:56:49.000Z,"Fox News Channel|""FNC""|""Fox News""|""News""|""Late...",99048,2231,151,1294,https://i.ytimg.com/vi/YQJmvXamKYg/default.jpg,False,False,False,The senior counselor to the president talks di...
30206,BOhylL90UVQ,18.14.06,The Chainsmokers - Somebody ft. Drew Love (Ror...,The Chainsmokers,10,2018-06-09T16:00:00.000Z,"The Chainsmokers Somebody|""Chainsmokers Somebo...",402345,32126,427,1895,https://i.ytimg.com/vi/BOhylL90UVQ/default.jpg,False,False,False,The Chainsmokers - Somebody ft. Drew Love (A R...


In [4]:
# Examining string for tags
df.tags[2]

'racist superman|"rudy"|"mancuso"|"king"|"bach"|"racist"|"superman"|"love"|"rudy mancuso poo bear black white official music video"|"iphone x by pineapple"|"lelepons"|"hannahstocking"|"rudymancuso"|"inanna"|"anwar"|"sarkis"|"shots"|"shotsstudios"|"alesso"|"anitta"|"brazil"|"Getting My Driver\'s License | Lele Pons"'

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30208 entries, 0 to 30207
Data columns (total 16 columns):
video_id                  30208 non-null object
trending_date             30208 non-null object
title                     30208 non-null object
channel_title             30208 non-null object
category_id               30208 non-null int64
publish_time              30208 non-null object
tags                      30208 non-null object
views                     30208 non-null int64
likes                     30208 non-null int64
dislikes                  30208 non-null int64
comment_count             30208 non-null int64
thumbnail_link            30208 non-null object
comments_disabled         30208 non-null bool
ratings_disabled          30208 non-null bool
video_error_or_removed    30208 non-null bool
description               29129 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 3.1+ MB


In [6]:
# Looking for NAN values
df.isna().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               1079
dtype: int64

In [7]:
df.shape

(30208, 16)

In [8]:
# Number of rows with no tags
len(df[df['tags'] == '[none]'])

1942

In [9]:
# Deleting rows with no tags
df = df[df['tags'] != '[none]']

In [10]:
df.shape

(28266, 16)

In [11]:
# Cleaning string for all tags in dataset
df['tags'] = df['tags'].str.replace(r'|', '')
df['tags'] = df['tags'].str.replace(r'"', '. ')
df['tags'] = df['tags'].str.replace(r' . ', ' ')
df['tags']

C:\Users\khan_\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\khan_\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\khan_\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


0                                          SHANtell martin
1        last week tonight trump presidency. last week ...
2        racist superman. rudy. mancuso. king. bach. ra...
3        rhett and link. gmm. good mythical morning. rh...
4        ryan. higa. higatv. nigahiga. i dare you. idy....
                               ...                        
30203    Netflix. Trailer. movies. streaming. movies on...
30204    5SOS. Seconds of Summer. No Roots. Cover. BBC....
30205    Fox News Channel. FNC. Fox News. News. Latest ...
30206    The Chainsmokers Somebody. Chainsmokers Somebo...
30207    5-Minute Crafts. DIY. Do it yourself. crafts. ...
Name: tags, Length: 28266, dtype: object

In [12]:
# Dropping unwanted columns
df.drop(columns=['publish_time', 'thumbnail_link', 'comments_disabled', 'ratings_disabled',
                 'video_error_or_removed', 'description'], inplace=True)

C:\Users\khan_\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
df.describe()

,category_id,views,likes,dislikes,comment_count
count,28266.000000,2.826600e+04,2.826600e+04,28266.000000,28266.000000
mean,20.982346,5.162653e+05,1.961761e+04,820.876424,2674.072419
std,6.720821,1.345758e+06,6.621552e+04,6499.775735,11510.251371
min,1.000000,5.490000e+02,0.000000e+00,0.000000,0.000000
25%,20.000000,8.855775e+04,1.456250e+03,63.000000,261.000000
50%,24.000000,2.152155e+05,4.907500e+03,178.000000,810.000000
75%,24.000000,4.960018e+05,1.542000e+04,494.000000,2110.000000
max,43.000000,4.843165e+07,3.880071e+06,629120.000000,733373.000000


In [33]:
to_drop = df[df['views'] > 1000000]
to_drop_index = to_drop.index
to_drop_index

Int64Index([   45,    50,    52,    55,    77,    78,    86,   100,   112,
              113,
            ...
            30092, 30117, 30120, 30128, 30131, 30140, 30177, 30186, 30190,
            30197],
           dtype='int64', length=1937)

In [40]:
df[df['views'] > 1000000]

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count


In [35]:
df.drop(index = to_drop_index, inplace=True)

In [36]:
df.shape

(25167, 10)

In [37]:
df.to_csv('Data/cleaned_df.csv', index=False)